Setup packages and read data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
ets = pd.read_csv('ETS_Database_v49_Jul22.csv', sep='\t')
ets.head()

,country,country_code,ETS information,main activity sector name,unit,value,year
0,Austria,AT,1.1 Freely allocated allowances,32 Manufacture of ceramics,tonne of CO2 equ.,498797.0,2005
1,Austria,AT,1.1 Freely allocated allowances,32 Manufacture of ceramics,tonne of CO2 equ.,383157.0,2013
2,Austria,AT,1.1 Freely allocated allowances,32 Manufacture of ceramics,tonne of CO2 equ.,351210.0,2017
3,Austria,AT,1.1 Freely allocated allowances,32 Manufacture of ceramics,tonne of CO2 equ.,335835.0,2018
4,Austria,AT,1.1 Freely allocated allowances,32 Manufacture of ceramics,tonne of CO2 equ.,299741.0,2021


In [3]:
ets.rename(columns={'ETS information':'info', 'main activity sector name':'sector'}, inplace=True)

Identify variables

In [4]:
rows_to_drop = {}
def add_rows_to_drop(colname, *rowvalues):
    if not colname in rows_to_drop: rows_to_drop[colname] = []
    rows_to_drop[colname].extend(rowvalues)

In [5]:
countries = ets[['country','country_code']].drop_duplicates()
add_rows_to_drop('country', 'NER 300 auctions')

In [6]:
info = ets['info'].unique()
info.sort()
add_rows_to_drop('info', '1. Total allocated allowances (EUA or EUAA)', '1.1 Freely allocated allowances', '2. Verified emissions', '4. Total surrendered units')


In [7]:
sectors = ets['sector'].unique()
add_rows_to_drop('sector', '21-99 All industrial installations (excl. combustion)', '20-99 All stationary installations')

In [8]:
units = ets['unit'].unique()
del ets['unit']

In [9]:
year = ets['year'].unique()
add_rows_to_drop('year', 'Total 1st trading period (05-07)', 'Total 2nd trading period (08-12)', 'Total 3rd trading period (13-20)', 'Total 4th trading period (21-30)')

drop rows:

In [10]:
for colname, values in rows_to_drop.items(): ets.drop(ets.loc[ets[colname].isin(values)].index, inplace=True)

In [13]:
ets.reset_index(inplace=True)
del ets['index']

In [14]:
hist_em = ets.copy()
hist_em

,level_0,country,country_code,info,sector,value,year
0,0,Austria,AT,1.1.1 Free allocation to existing entities (Ar...,32 Manufacture of ceramics,481340.0,2011
1,1,Austria,AT,1.1.1 Free allocation to existing entities (Ar...,32 Manufacture of ceramics,478462.0,2012
2,2,Austria,AT,1.1.2 Free allocation from the new entrants re...,32 Manufacture of ceramics,0.0,2017
3,3,Austria,AT,1.1.2 Free allocation from the new entrants re...,32 Manufacture of ceramics,0.0,2018
4,4,Austria,AT,1.1.2 Free allocation from the new entrants re...,32 Manufacture of ceramics,0.0,2021
...,...,...,...,...,...,...,...
33049,33049,Slovenia,SI,2.1 EU-ETS Verified Emission,24 Production of pig iron or steel,217488.0,2016
33050,33050,Slovenia,SI,2.1 EU-ETS Verified Emission,24 Production of pig iron or steel,222518.0,2019
33051,33051,Slovenia,SI,4.2 Surrendered certified emission reductions ...,24 Production of pig iron or steel,0.0,2011
33052,33052,Slovenia,SI,4.2 Surrendered certified emission reductions ...,24 Production of pig iron or steel,0.0,2012


In [16]:
EU27 = ['Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus', 'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Greece', 'Hungary', 'Ireland', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Malta', 'Netherlands', 'Poland', 'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'Spain', 'Sweden']
UK = ['United Kingdom (excl. NI)', 'Northern Ireland']
EU27_UK = [EU27, UK]